# TFM Máster Data Science UAH 2020-2021

### MIGUEL PÉREZ CARO

Este notebook tiene como objetivo el análisis de un set de datos obtenido a través de Twitter en Febrero de 2015 que recoge tweets que hacen referencia a las principales aerolíneas estadounidenses y cuyo sentimiento ha sido clasificado manualmente. El set de datos se encuentra en: https://www.kaggle.com/crowdflower/twitter-airline-sentiment

EL objetivo es comprobar dicho sentimiento y las diferencias que se encuentran con los resultados obtenidos en el dataset que se ha recogido durante el verano de 2021.

En primer lugar se importan las librerías

In [58]:
import os
import re
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from emot.emo_unicode import EMOTICONS_EMO, UNICODE_EMOJI
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Se establece el directorio de trabajo.

In [2]:
directorio = os.path.dirname(os.getcwd())
mydir = os.path.join(directorio, 'data')
os.chdir(mydir)

Se carga el archivo.

In [3]:
df = pd.read_csv('kaggle.csv')

Se analiza el dataset.

In [7]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
df.shape

(14640, 15)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [8]:
df.airline.unique()

array(['Virgin America', 'United', 'Southwest', 'Delta', 'US Airways',
       'American'], dtype=object)

Solo interesan los campos:

- airline_sentiment
- negativereason
- airline
- text

Y las aerolíneas analizadas en el estudio que son: American, Delta, Southwest y United

In [9]:
df.drop(columns=['tweet_id', 'airline_sentiment_confidence', 'negativereason_confidence', 'airline_sentiment_gold',
                 'name', 'negativereason_gold', 'retweet_count', 'tweet_coord', 'tweet_created', 'tweet_location',
                 'user_timezone'], inplace = True)

,airline_sentiment,negativereason,airline,text
0,neutral,NaN,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,NaN,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,NaN,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,Bad Flight,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,Can't Tell,Virgin America,@VirginAmerica and it's a really big bad thing...


In [11]:
df = df[df['airline'].isin(['United', 'Southwest', 'Delta', 'American'])]

In [12]:
df.airline.unique()

array(['United', 'Southwest', 'Delta', 'American'], dtype=object)

Se va a comprobar la distribución del sentimiento de los tweets por aerolínea:

In [19]:
print("Cantidad total de tweets recogidos por aerolínea:\n")
print(df.groupby('airline')['text'].count())

Cantidad total de tweets recogidos por aerolínea:

airline
American     2759
Delta        2222
Southwest    2420
United       3822
Name: text, dtype: int64


In [45]:
airlines= ['United','American','Southwest','Delta']
columns1 = ['Positivo', 'Neutral', 'Negativo', 'Total']
columns2 = ['Positivo', 'Neutral', 'Negativo']

data_total = list()
data_porcentaje = list()

for i in airlines :

    data1 = [list(df.loc[df.airline == i].airline_sentiment.value_counts().sort_index(ascending = False).values)+[df.loc[df.airline == i].shape[0]]]
    data_total = data_total+data1
    
    data2 = np.array(df.loc[df.airline == i].airline_sentiment.value_counts().sort_index(ascending = False).values)/df.loc[df.airline == i].shape[0]
    data2 = np.around(data2, decimals=4)*100
    data_porcentaje.append(data2)  

In [48]:
df_total = pd.DataFrame(data_total, columns = columns1, index = airlines)
df_total

,Positivo,Neutral,Negativo,Total
United,492,697,2633,3822
American,336,463,1960,2759
Southwest,570,664,1186,2420
Delta,544,723,955,2222


In [49]:
df_porcentaje = pd.DataFrame(data_porcentaje, columns = columns2, index = airlines)
df_porcentaje

,Positivo,Neutral,Negativo
United,12.87,18.24,68.89
American,12.18,16.78,71.04
Southwest,23.55,27.44,49.01
Delta,24.48,32.54,42.98


Una vez mostrados los resultados en cuanto al sentimiento por aerolínea, también interesa obtener los diferentes motivos a los que se debe el sentimiento negativo, lo cuál puede ser comparado con los patrones encontrados:

In [50]:
print('Razones por las que un tweet se ha considerado como algo negativo:\n')
print(df.negativereason.value_counts())

Razones por las que un tweet se ha considerado como algo negativo:

Customer Service Issue         2039
Late Flight                    1195
Can't Tell                      922
Cancelled Flight                640
Lost Luggage                    565
Bad Flight                      457
Flight Booking Problems         379
Flight Attendant Complaints     353
longlines                       125
Damaged Luggage                  59
Name: negativereason, dtype: int64


Por curiosidad, se va a aplicar la limpieza realizada y a utilizar el analizador de Sentimientos VADER para ver si los resultados difieren en gran medida de los obtenidos.

Se comienza por la limpieza

In [51]:
def clean_emoticons(text):
    """
    Función generada para hacer la limpieza de emoticonos.
    
    param: text texto a limpiar
    :return: texto limpio
    """
    for emot in EMOTICONS_EMO:
        if emot in text:
            text = text.replace(emot, " {} ".format(EMOTICONS_EMO[emot]))
    return text

In [52]:
def clean_emojis(text):
    """
    Función generada para hacer la limpieza de emojis.
    
    param: text texto a limpiar
    :return: texto limpio
    """
    for emot in UNICODE_EMOJI:
        if emot in text:
            text = text.replace(emot, " {} ".format(UNICODE_EMOJI[emot].replace(':',"").replace('_',' ')))
    return text

In [54]:
lista_elementos_eliminar = ['#', ' rt ', '\n', 'american airlines', 'delta airlines', 'united airlines', 
                            'southwest airlines', ' aa ', ' aal ', ' dl ', ' dal ', ' ua ', ' ual ', ' swa ',
                            ' mi ', ' ft ', ' frm ', ' hrzn ', ' amp ']

In [55]:
def cleaning_text_basico(df, lista_elementos_eliminar):
    """
    Función generada para hacer una limpieza general del texto.
    
    param: df dataframe a limpiar
    :return: lista con el texto limpiado. Un elemento por cada fila del dataframe.
    """
    corpus = [' ' + text + ' ' for text in  df.text.tolist()] # Añadir espacio al principio y al final apra fcilitar limpieza
    corpus = [re.sub('@[A-Za-z0-9_]+',' ', text) for text in corpus] # Eliminar usuarios
    corpus = [re.sub(r'http\S+', '', text) for text in corpus] # Eliminas enlaces web
    corpus = [text.lower() for text in corpus] # Pasar a minúsculas
    #corpus = [clean_emojis(text) for text in corpus] # Eliminar emojis
    #corpus = [clean_emoticons(text) for text in corpus] # Eliminar emoticonos
    #corpus = [''.join(ch for ch in text if ch not in string.punctuation) for text in corpus] # Eliminar signos de puntuación
    corpus = [re.sub(r'[^a-z\s]', '', text) for text in corpus] # Eliminar lo que no sea minúscula o espacio
    for i in lista_elementos_eliminar:
        corpus = [re.sub(i,' ', text) for text in corpus] # Eliminar elementos de la lista
    corpus = [text.strip() for text in corpus] # Normalizar espacios
    corpus = [re.sub(r' +', ' ', text) for text in corpus] # Normalizar espacios
    corpus = [' '.join(word for word in text.split() if word not in stopwords.words('english')) for text in corpus] # Eliminar stopwords 
    return corpus

In [56]:
corpus_clean = cleaning_text_basico(df, lista_elementos_eliminar)

In [57]:
df['text_clean'] = corpus_clean

Se procede a realizar el análisis de sentimiento

In [61]:
vader_analyzer = SentimentIntensityAnalyzer()

In [66]:
def vader_sentiment_analysis(text):
    """
    Función para calcular el sentimiento de los tweets a partir de la librería VADER.
    
    param: text texto a calcular su sentimiento.
    :return: vader_sentiment resultado compound del análisis.
    """
    vader_sentiment_full = vader_analyzer.polarity_scores(text)
    vader_sentiment = vader_sentiment_full['compound']
    return vader_sentiment

In [67]:
df['vader_sentiment'] = df['text_clean'].apply(vader_sentiment_analysis)

In [68]:
def vader_sentiment_texto(score): 
    """
    Función para pasar a texto el resultado del análisis de sentimiento.
    
    param: score el resultado del análisis.
    :return: Valor en texto del resultado
    """
    if score < -0.05:
        return 'Negativo'
    elif score > 0.05:
        return 'Positivo'
    else:
        return 'Neutral'

In [69]:
df['vader_sentiment_analysis'] = df['vader_sentiment'].apply(vader_sentiment_texto)

Se obtienen los valores totales y porcentuales como se hizo anteriormente

In [70]:
data_total_vader = list()
data_porcentaje_vader = list()

for i in airlines :
    data1 = [list(df.loc[df.airline == i].vader_sentiment_analysis.value_counts().sort_index(ascending = False).values)+[df.loc[df.airline == i].shape[0]]]
    data_total_vader = data_total_vader+data1
    
    data2 = np.array(df.loc[df.airline == i].vader_sentiment_analysis.value_counts().sort_index(ascending = False).values)/df.loc[df.airline == i].shape[0]
    data2 = np.around(data2, decimals=4)*100
    data_porcentaje_vader.append(data2)  
    

In [73]:
data_total_vader = pd.DataFrame(data_total_vader, columns = columns1, index = airlines)

In [74]:
data_porcentaje_vader = pd.DataFrame(data_porcentaje_vader, columns = columns2, index = airlines)

In [75]:
df_total

,Positivo,Neutral,Negativo,Total
United,492,697,2633,3822
American,336,463,1960,2759
Southwest,570,664,1186,2420
Delta,544,723,955,2222


In [76]:
data_total_vader

,Positivo,Neutral,Negativo,Total
United,1666,878,1278,3822
American,1153,665,941,2759
Southwest,1213,611,596,2420
Delta,1015,694,513,2222


In [78]:
df_porcentaje

,Positivo,Neutral,Negativo
United,12.87,18.24,68.89
American,12.18,16.78,71.04
Southwest,23.55,27.44,49.01
Delta,24.48,32.54,42.98


In [79]:
data_porcentaje_vader

,Positivo,Neutral,Negativo
United,43.59,22.97,33.44
American,41.79,24.10,34.11
Southwest,50.12,25.25,24.63
Delta,45.68,31.23,23.09
